### TF KOMPAS: Gapped Dimeric Calls

Author: Zachery Mielko

Multi-core calls will call TFBS that are palindromic dimers with a significant gap in the middle that makes calling over one large site difficult with shorter kmers relative to the length of the model. The script uses KOMPAS twice over both cores, then calls sites that are a certain distance away.

Requires:
1. TF_KOMPAS_Site_Caller.py in the same directory 
2. Pandas, pybedtools, and os (all of which are required in TF_KOMPAS_Site_Caller.py)

Input:
1. Peak File (bed)
2. Genome File
3. Core parameters
4. Threshold Parameters
5. Gap distance


**Note**

The distance used in multi-core calls should (ideally) be the same distance as in the model for palindromic dimers. If a transcription factor has the potential for a variety of distances, alignments with models for each distance should be used independently. Your mileage may vary. 




In [22]:
import pandas as pd
import os
from pybedtools import BedTool


##### parameters #####
# coreL and coreR are relative to the position of the model
coreL = (9,14 )
coreR = (17,22 )
threshold = 0.4
gapDistance = 3
logFile = True
# Files and folders
wkdir = "/Users/ZMielko/Desktop/Melanoma_Paper/TopGeneAnalysis2/NR3C1/"
peakFile = '/Users/ZMielko/Desktop/Melanoma_Paper/TopGeneAnalysis2/NR3C1/NR3C1_hg19DHSPromoterNoDac.bed'
genomeFile = '/Users/ZMielko/Desktop/In_Vivo_Project/Data/human_g1k_v37.fasta'
kmerFile = '/Users/ZMielko/Desktop/Melanoma_Paper/TopGeneAnalysis2/NR3C1/NR3C1_8mers_aligned.txt'
KOMPAS = f'{os.getcwd}/TF_KOMPAS_Site_Caller.py'

# Run KOMPAS Site Caller and rename output for each core
os.system(f"python {KOMPAS} {wkdir} {peakFile} {genomeFile} {kmerFile} {coreL[0]} {coreL[1]} {threshold} -isPalindrome True -logFile True -centerPos {coreL[1]-1}")
os.system(f"mv {wkdir}Centered_{threshold}_KOMPAS.bed {wkdir}Centered_{threshold}_Left.bed")
os.system(f"python {KOMPAS} {wkdir} {peakFile} {genomeFile} {kmerFile} {coreR[0]} {coreR[1]} {threshold} -isPalindrome True -logFile True -centerPos {coreR[0]}")
os.system(f"mv {wkdir}Centered_{threshold}_KOMPAS.bed {wkdir}Centered_{threshold}_Right.bed")

# Determine distance between cores edges is equal to specified distance. 
sitesL = pd.read_csv(f"{wkdir}Centered_{threshold}_Left.bed", sep = '\t', header = None)
sitesR = pd.read_csv(f"{wkdir}Centered_{threshold}_Right.bed", sep = '\t', header = None)
sitesL[1] = sitesL[1] + (gapDistance + 1)
sitesL[2] = sitesL[2] + (gapDistance + 1)
bedL = BedTool.from_dataframe(sitesL)
bedR = BedTool.from_dataframe(sitesR)
fullSites = bedL.intersect(bedR)
fullSites.to_dataframe().to_csv(f"{wkdir}Centered_dimeric_d{distance}.bed", sep = '\t', header = None, index = False)

if logFile == True:
    ##################
    # Log the output #
    ##################
    f = open(wkdir + "/KOMPASLog.txt", "a")
    f.write("##### Parameters ##### \n")
    f.write(f"Peak file: {peakFile}"+ "\n")
    f.write(f"Genome file: {genomeFile}"+ "\n")
    f.write(f"kmer file: {kmerFile}"+ "\n")
    f.write(f"Core kPositions: {coreL} {coreR}"+ "\n")
    f.write(f"Threshold: {threshold}"+ "\n")
    f.write("#Summary# \n")
    f.write(f"Final # of called sequences: {len(fullSites)}"+ "\n")
    f.close()


0